<a href="https://www.kaggle.com/code/lonnieqin/islr-create-tfrecord?scriptVersionId=120839993" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

## ISLR: Create TFRecord

In this notebook, I will create time-series TFRecord for [Google - Isolated Sign Language Recognition Competition](https://www.kaggle.com/competitions/asl-signs). I will choose 12 as sequence_length. The dataset has dynamic sequence length, I will convert to static sequence length by simply using `tf.image.resize` API. [Here](https://www.kaggle.com/code/lonnieqin/isolated-sign-language-recognition-with-convlstm1d/notebook?scriptVersionId=120834686) is a baseline to train with this dataset. 

## Configuration

In [ ]:
class CFG:
    data_path = "../input/asl-signs/"
    sequence_length = 12
    rows_per_frame = 543 

## Import Libraries

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tqdm import tqdm
import json
import os

## Utilities

In [ ]:
ROWS_PER_FRAME = 543  # number of landmarks per frame

def load_relevant_data_subset_with_imputation(pq_path):
    data_columns = ['x', 'y', 'z']
    data = pd.read_parquet(pq_path, columns=data_columns)
    data.replace(np.nan, 0, inplace=True)
    n_frames = int(len(data) / ROWS_PER_FRAME)
    data = data.values.reshape(n_frames, ROWS_PER_FRAME, len(data_columns))
    return data.astype(np.float32)

def load_relevant_data_subset(pq_path):
    data_columns = ['x', 'y', 'z']
    data = pd.read_parquet(pq_path, columns=data_columns)
    n_frames = int(len(data) / ROWS_PER_FRAME)
    data = data.values.reshape(n_frames, ROWS_PER_FRAME, len(data_columns))
    return data.astype(np.float32)

def read_dict(file_path):
    path = os.path.expanduser(file_path)
    with open(path, "r") as f:
        dic = json.load(f)
    return dic

## Load data

In [ ]:
train = pd.read_csv(f"{CFG.data_path}train.csv")
label_index = read_dict(f"{CFG.data_path}sign_to_prediction_index_map.json")
index_label = dict([(label_index[key], key) for key in label_index])
print(label_index)
train["label"] = train["sign"].map(lambda sign: label_index[sign])
train.head()

## Create TF-Record

In [ ]:
def create_record(feature, label):
    dic = {}
    dic["feature"] = tf.train.Feature(float_list=tf.train.FloatList(value=feature))
    dic["label"] = tf.train.Feature(int64_list=tf.train.Int64List(value=[label]))
    record_bytes = tf.train.Example(features=tf.train.Features(feature=dic)).SerializeToString()
    return record_bytes
    
def decode_function(record_bytes):
  return tf.io.parse_single_example(
      # Data
      record_bytes,
      # Schema
      {
          "feature": tf.io.FixedLenFeature([CFG.sequence_length * CFG.rows_per_frame * 3], dtype=tf.float32),
          "label": tf.io.FixedLenFeature([], dtype=tf.int64)
      }
  )

In [ ]:
for participant_id in train.participant_id.unique():
    df = train[train.participant_id == participant_id]
    save_path = f"{participant_id}.tfrecords"
    with tf.io.TFRecordWriter(save_path) as file_writer:
        for i in tqdm(range(len(df))):
            path = f"{CFG.data_path}{df.iloc[i].path}"
            feature = load_relevant_data_subset_with_imputation(path)
            feature = tf.image.resize(tf.constant(feature), (CFG.sequence_length, 543)).numpy().reshape(-1)
            label = int(df.iloc[i].label)
            file_writer.write(create_record(feature, label))